### Cloud Creating a GKE Cluster<br>

***

<br>

We'll be building and deploying our own containerized application in the cluster<br>

**Demo**<br>

Google Cloud -> Kubernetes Engine -> Create a cluster<br>
name: cluster1<br>
location type: zonal, us-east1-b<br>
master version: release channel: regular channel(default), version: 1.17.12-gke.1504(default)<br>

Select default-pool in the sidebar<br>
click on autoscaling: min:1, max:3<br>

default-pool -> nodes<br>
image type: Container Optimized OS (COS) (default)<br>
series: E2, e2-micro<br>
boot disk size: 10GB<br>

Select cluster -> networking<br>
enable HTTP loadbalancing - add on that's required in order to use Google's loadbalancer<br>
<br>

Cloud Shell<br>

In [ ]:
# to create a cluster through command line
gcloud container clusters create cluster1 --num-nodes 3
# but if you want to build exactly like the one, it's more long due to details of it

# in order to interact with cluster, you need to use kubectl
gcloud container clusters get-credentials cluster1 --zone us-east1-b
# ^- kubectl has been configured

kubectl get pods
kubectl get nodes # check if it's configured correctly

Will be building a docker image using Cloud build<br>
Will be then pushed to Google's Container registry<br>
to deploy to current GKE cluster<br>

Clone your repo within the Cloud shell<br>
to run necessary commands to build the image<br>

In [ ]:
cd ~
git clone https://github.com/antonitz/google-cloud-associate-cloud-engineer.git
cd google-cloud-associate-cloud-engineer/
cd 09-Kubernetes-Engine-and-Containers/box_of_bowties(production1)/container
# the container contains 3 files: img, docker file and an html

# will be using cloud build: serverless CI/CD platform that allows me to package source code into containers
# we won't be doing CI/CD pipeline, but just to build an image and push to the container

# enable the cloud build API
gcloud services enable cloudbuild.googleapis.com

Google Cloud -> Tools -> Cloud Build<br>
To observe what it's doing<br>

In [ ]:
gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/production1:1.0.0 .

after build is complete, you can check the summary at 'History' tab in Cloud Build<br>
you can click on view 'gcr.io/bowtieinc/production1:1.0.0 image'<br>

It'd open up the container registry<br>
you can deploy to Cloud Run/GKE/GCE with a click on button, but will do it in GKE<br>

Google Cloud -> Kubernetes Engine -> Click on Deploy<br>
Select image path -> select the image<br>
app name: production1<br>
labels: key: app, value: production1<br>
cluster: cluster1(us-east1-b)<br>
<br>

Kubernetes engine show live representation of my deployment<br>

In [ ]:
kubectl get all # see all the details
kubectl get deployments # shows the deployment
kubectl get pods # shows the names of pods
kubectl describe pod [NAME_OF_THE_POD] # to see all the events in the pod

Click on 'Expose' deployment<br>
port mapping: target port: 80<br>
service type: loadbalancer<br>
service name: service1<br>
<br>

Kubernetes Engine -> Services & Ingress<br>
click on endpoint address -> click the redirect url<br>
this will open up the endpoint HTML static page you've created<br>
<br>

In [ ]:
kubectl get pods
kubectl scale deployment production1 --replicas=1
kubectl get pods # only 1 pod running in the workload
kubectl scale deployment production1 --replicas=3
kubectl get pods # back to 3 replicas

# to edit
cd ~
cd 09-Kubernetes-Engine-and-Containers/box_of_bowties(production1)/container
edit index.html # this will open up the editor
# edit some text and save it
gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/production1:1.0.1 . # 1.0.1 different version

Google Cloud -> Cloud Build -> Build Artifact<br>
click on view image and verify everything is ok<br>

Google Cloud -> Kubernetes Engine -> Workloads -> Production1<br>
Actions -> rolling update<br>
open a new tab -> Container Registry -> Digest Detail -> SHA256 hash copy<br>
back to the rolling update, replace the SHA256 hash values<br>
This will schedule an update for your application<br>

In [ ]:
kubectl get pods # new pods have started, by the age of the pods
